In [3]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

# Get the data and create the output directory (Extraction)

In [1]:
path_dir_data = "/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/data"

In [4]:
# Obtenemos la fecha actual para crear un directorio
# donde se guardaran los archivos de salida
date_now = datetime.now().strftime("%d_%m_%Y") 
path_dir_today =  os.path.join(path_dir_data, date_now)

# Creamos el directorio si no existe
os.makedirs(path_dir_today, exist_ok=True)

In [5]:
query_productos_sims = """ 
WITH CLIENTES_ACTIVOS AS (
    SELECT CLAVE, NOMBRE
    FROM SIMS.SIMS_CONTRAPARTES 
    WHERE NOMBRE NOT LIKE '%INACTI%' AND NOMBRE NOT LIKE '%APLICA%'
), MOVIMIENTO AS (
    SELECT SM.*, CAT.DESCRIPCION MOVIMIENTO
    FROM SIMS.SIMS_MOVIMIENTOS SM
    LEFT JOIN SIMS.SIMS_TIPOS_MOVTOS CAT ON CAT.CLAVE = SM.TIPO_MOVTO
),  MOVIMIENTO_C2 AS (
    SELECT SM.*, CAT.DESCRIPCION PORTAFOLIO
    FROM MOVIMIENTO SM
    LEFT JOIN SIMS.SIMS_TIPOS_PORTAFOLIOS CAT ON CAT.TIPO = SM.TIPO_PORTAFOLIO
), MOVIMENTO_C3 AS (
	SELECT SM.*, CAT.DESCRIPCION VAR_MOVIMIENTO
    FROM MOVIMIENTO_C2 SM
    LEFT JOIN SIMS.SIMS_VARIANTES_MOVTOS CAT ON ( CAT.TIPO = SM.TIPO_MOVTO  AND CAT.CLAVE = SM.VARIANTE_MOVTO)
), TAB_FINAL AS (
	SELECT CLAVE,NOMBRE,FOLIO,TIPO_MOVTO,MOVIMIENTO, TIPO_PORTAFOLIO, PORTAFOLIO, VARIANTE_MOVTO, VAR_MOVIMIENTO, FECHA_OPERACION,FECHA_VALOR,NUMERO_EMISION,
	FECHA_CUPON_ACTUAL,TASA,DIAS,PRECIO,VALOR_NOMINAL,VALOR_NOMINAL_AMORTIZADO,VALOR_COSTO,INTERESES_ACUMULADOS,
	MONTO_MOVIMIENTO,MONTO_VRR,VALOR_NOMINAL_DISPONIBLE,INTERESES_ANTICIPADOS,INTERESES_PROVISIONADOS,INTERESES_COMPLEMENTO,
	UTILIDAD,PERDIDA,FOLIO_COMPRA,SUBFOLIO_COMPRA,ESTATUS_MOVTO,TIPO_CAMBIO_COMPRA,TIPO_CAMBIO_VENTA,TITULOS_VENCIMIENTO,
	FECHA_LIQUIDA,NUMERO_TITULOS,PRECIO_SUCIO,FECHA_LIQUIDACION
	FROM CLIENTES_ACTIVOS AC
	LEFT JOIN MOVIMENTO_C3 MV ON MV.CLAVE_CONTRAPARTE = AC.CLAVE
) SELECT * FROM TAB_FINAL
"""

In [6]:
path_dir_productos = get_table(path_dir_today, db_sims, "productos_clientes_sims", query = query_productos_sims)

# Transform the data and output the transformation

In [8]:
path_dir_output_transformation = os.path.join("/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/agrupados", date_now)
os.makedirs(path_dir_output_transformation, exist_ok=True)
path_dir_output_transformation

'/home/ale1726/proyects/datalake/clientes/data/productos/SIMS/agrupados/21_04_2025'

In [9]:
productos_sims =  pd.read_csv(path_dir_productos, low_memory=False)

In [11]:
productos_sims["FECHA_OPERACION"] = pd.to_datetime(productos_sims["FECHA_OPERACION"])


In [39]:
productos_sims["AÑO_FOPE"] = productos_sims["FECHA_OPERACION"].dt.year.astype('Int64')
productos_sims["MES_FOPE"] = productos_sims["FECHA_OPERACION"].dt.month
productos_sims["MES_FOPE_STR"] = productos_sims["FECHA_OPERACION"].dt.month_name()        

#productos_meca["MES"] = productos_meca["OPE_FECHA_ALTA"].dt.monthv

In [31]:
productos_sims.columns

Index(['CLAVE', 'NOMBRE', 'FOLIO', 'TIPO_MOVTO', 'MOVIMIENTO',
       'TIPO_PORTAFOLIO', 'PORTAFOLIO', 'VARIANTE_MOVTO', 'VAR_MOVIMIENTO',
       'FECHA_OPERACION', 'FECHA_VALOR', 'NUMERO_EMISION',
       'FECHA_CUPON_ACTUAL', 'TASA', 'DIAS', 'PRECIO', 'VALOR_NOMINAL',
       'VALOR_NOMINAL_AMORTIZADO', 'VALOR_COSTO', 'INTERESES_ACUMULADOS',
       'MONTO_MOVIMIENTO', 'MONTO_VRR', 'VALOR_NOMINAL_DISPONIBLE',
       'INTERESES_ANTICIPADOS', 'INTERESES_PROVISIONADOS',
       'INTERESES_COMPLEMENTO', 'UTILIDAD', 'PERDIDA', 'FOLIO_COMPRA',
       'SUBFOLIO_COMPRA', 'ESTATUS_MOVTO', 'TIPO_CAMBIO_COMPRA',
       'TIPO_CAMBIO_VENTA', 'TITULOS_VENCIMIENTO', 'FECHA_LIQUIDA',
       'NUMERO_TITULOS', 'PRECIO_SUCIO', 'FECHA_LIQUIDACION', 'AÑO_FOPE',
       'MES_FOPE', 'MES_FOPE_STR'],
      dtype='object')

In [71]:
productos_sims.value_counts("NUMERO_TITULOS")

NUMERO_TITULOS
1000.0       212
2000.0       122
5000.0        99
10000.0       53
500.0         44
            ... 
3750.0         1
3938.0         1
4023.0         1
23.0           1
2500000.0      1
Name: count, Length: 61, dtype: int64

In [76]:
agrupado_test = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        descripcion=('NUMERO_TITULOS', 'sum'), 
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [94]:
agrupado_portafolio = (
    productos_sims.groupby(['PORTAFOLIO'], as_index=False)
    .agg(
        valor=('FOLIO', 'count'), 
    ).sort_values(by=['PORTAFOLIO'], ascending=True)
)

In [95]:
agrupado_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_portafolio.csv"), index=False)

In [89]:
agrupado_año_portafolio = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        valor=('PORTAFOLIO', 'nunique'), 
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [93]:
agrupado_año_portafolio.to_csv(os.path.join(path_dir_output_transformation, "agrupado_año_portafolio.csv"), index=False)

In [20]:
agrupado_operaciones_anuales = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        num_operaciones=('FOLIO', 'count'), 
        monto_anuales_sum=('MONTO_MOVIMIENTO', 'sum'), 
        monto_anuales_min=('MONTO_MOVIMIENTO', 'min'), 
        monto_anuales_max=('MONTO_MOVIMIENTO', 'max'), 
        monto_anuales_mean=('MONTO_MOVIMIENTO', 'mean'), 
        monto_anuales_median=('MONTO_MOVIMIENTO', 'median')
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [21]:
agrupado_operaciones_anuales.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales.csv"), index=False)

In [23]:
agrupado_operaciones_anuales_mes = (
    productos_sims.groupby(['AÑO_FOPE','MES_FOPE','MES_FOPE_STR'], as_index=False)
    .agg(
        num_operaciones=('FOLIO', 'count'), 
        monto_anuales_sum=('MONTO_MOVIMIENTO', 'sum'), 
        monto_anuales_min=('MONTO_MOVIMIENTO', 'min'), 
        monto_anuales_max=('MONTO_MOVIMIENTO', 'max'), 
        monto_anuales_mean=('MONTO_MOVIMIENTO', 'mean'), 
        monto_anuales_median=('MONTO_MOVIMIENTO', 'median')
    ).sort_values(by=['AÑO_FOPE','MES_FOPE'], ascending=True)
)

In [24]:
agrupado_operaciones_anuales_mes.to_csv(os.path.join(path_dir_output_transformation, "agrupado_operaciones_anuales_mes.csv"), index=False)

In [ ]:
agrupado_plazo_anual = (
    productos_sims.groupby(['AÑO_FOPE'], as_index=False)
    .agg(
        plazo_mean=('DIAS', lambda x: round(x.mean())),
        plazo_min=('DIAS', 'min'), 
        plazo_max=('DIAS', 'max'),  
        plazo_median=('DIAS', 'median'),
        utilidad = ('UTILIDAD', 'sum'),
        perdida = ('PERDIDA', 'sum'),
    ).sort_values(by=['AÑO_FOPE'], ascending=True)
)

In [44]:
agrupado_plazo_anual.to_csv(os.path.join(path_dir_output_transformation, "agrupado_plazo_anual.csv"), index=False)

In [96]:
productos_sims

,CLAVE,NOMBRE,FOLIO,TIPO_MOVTO,MOVIMIENTO,TIPO_PORTAFOLIO,PORTAFOLIO,VARIANTE_MOVTO,VAR_MOVIMIENTO,FECHA_OPERACION,...,TIPO_CAMBIO_COMPRA,TIPO_CAMBIO_VENTA,TITULOS_VENCIMIENTO,FECHA_LIQUIDA,NUMERO_TITULOS,PRECIO_SUCIO,FECHA_LIQUIDACION,AÑO_FOPE,MES_FOPE,MES_FOPE_STR
0,29653,NAFIN/CEDEL,6317.0,E,EGRESO,100.0,IFN,1.0,EGRESO DE BONOS CON PAGO DE EFECTIVO VIA CEDEL,2008-10-28,...,NaN,NaN,NaN,2008-10-30 10:24:00,NaN,NaN,NaN,2008,10.0,October
1,NAFSEC,"NAFIN SECURITIES, NEW YORK",6304.0,E,EGRESO,100.0,IFN,1.0,EGRESO DE BONOS CON PAGO DE EFECTIVO VIA CEDEL,2007-10-25,...,NaN,10.9,NaN,2007-10-25 14:22:00,NaN,NaN,NaN,2007,10.0,October
2,NAFSEC,"NAFIN SECURITIES, NEW YORK",6303.0,I,INGRESO,100.0,IFN,1.0,INGRESO DE BONOS CON PAGO DE EFECTIVO VIA CEDEL,2007-10-25,...,NaN,NaN,NaN,2007-10-25 11:31:00,NaN,NaN,NaN,2007,10.0,October
3,NAFSEC,"NAFIN SECURITIES, NEW YORK",6302.0,E,EGRESO,100.0,IFN,1.0,EGRESO DE BONOS CON PAGO DE EFECTIVO VIA CEDEL,2007-10-24,...,NaN,10.9,NaN,2007-10-25 12:05:00,NaN,NaN,NaN,2007,10.0,October
4,NAFSEC,"NAFIN SECURITIES, NEW YORK",6301.0,E,EGRESO,100.0,IFN,1.0,EGRESO DE BONOS CON PAGO DE EFECTIVO VIA CEDEL,2007-10-24,...,NaN,10.9,NaN,2007-10-25 12:05:00,NaN,NaN,NaN,2007,10.0,October
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608,MONEXM,MONEX CASA DE BOLSA C.A. DE C.V.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
5609,8326,"INDEVAL, NAFIN",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
5610,FINMX,CASA DE BOLSA FINAMEX S.A.B. DE C.V.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN
5611,VECTOR,VECTORGLOBAL WMG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN


In [125]:
agrupado_clientes_año = (
    productos_sims.groupby(['CLAVE','NOMBRE', 'AÑO_FOPE'], as_index=False)
    .agg(
        transacciones=('FOLIO', lambda x: round(x.sum())),
        num_productos=('PORTAFOLIO', 'nunique'),
        monto_mean=('MONTO_MOVIMIENTO', lambda x: round(x.mean(), 2) if x.mean() != 0 else 0),
        plazo_promedio=('DIAS', lambda x: f"{round(x.mean())} Días"),
        tasa_prom=('TASA', lambda x: f"{round(x.mean(),2)}%"),
    )
    .sort_values(by=['AÑO_FOPE'], ascending=False)
)


In [127]:
agrupado_clientes_año.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_año.csv"), index=False)

In [126]:
agrupado_clientes_año

,CLAVE,NOMBRE,AÑO_FOPE,transacciones,num_productos,monto_mean,plazo_promedio,tasa_prom
266,MSTAN,"MORGAN STANLEY & CO. INCORPORATED, NEW YORK",2025,7072,1,2.000000e+06,0 Días,6.88%
186,GOLDM,GOLDMAN SACHS & CO. LLC,2025,14144,1,7.511651e+06,4 Días,6.88%
269,NAFIN,NACIONAL FINANCIERA SNC,2025,28302,1,5.016189e+06,9 Días,6.88%
196,HSBC.,HSBC MEXICO,2025,7070,1,7.019250e+06,0 Días,6.88%
50,BCOMXT,"BANCO NACIONAL DE COMERCIO EXTERIOR, S.N.C.",2025,28306,1,1.675000e+08,0 Días,4.78%
...,...,...,...,...,...,...,...,...
173,GOLDM,GOLDMAN SACHS & CO. LLC,1996,2598,1,4.992600e+06,0 Días,10.75%
126,CREDIT,"CREDIT LYONEE,NEW YORK",1996,2819,1,1.120014e+06,7 Días,10.0%
30,BANKTR,"BANKERS TRUST COMPANY, NEW YORK",1996,2527,1,5.500694e+05,20 Días,13.75%
327,SWISS,"SWISS BANK, NEW YORK",1995,10018,1,6.448120e+03,89 Días,6.25%


In [148]:
agrupado_clientes_tra_monto = (
    productos_sims.groupby(['NOMBRE'], as_index=False)
    .agg(
        transacciones=('FOLIO', lambda x: round(x.sum())),
        montos=('MONTO_MOVIMIENTO', lambda x: round(x.mean(), 2))
    )
    .sort_values(by='NOMBRE', ascending=False)
)


In [152]:
agrupado_clientes_tra_monto = agrupado_clientes_tra_monto[agrupado_clientes_tra_monto["transacciones"] >0].dropna()

In [153]:
agrupado_clientes_tra_monto

,NOMBRE,transacciones,montos
90,"WEST MERCHANT, NEW YORK",614614,9247906.94
89,WEST LANDESBANK,6702327,13281730.29
88,VESTRUST SEC.,36494,1483176.77
86,ULTRALAT CORP US,12653,3022127.00
85,UBS AG,5599973,3393453.57
...,...,...,...
4,BANCO INBURSA,7049,12092500.00
3,BANCO BILVAO VISCAYA,2069166,2231380.21
2,"BANCA SERFIN SA , NEW YORK",11550340,0.00
1,AJG INVESTMENTS HONG KONG LTD (MERRILL LYNCH),900030,0.00


In [154]:
agrupado_clientes_tra_monto.to_csv(os.path.join(path_dir_output_transformation, "agrupado_clientes_tra_monto.csv"), index=False)